In [92]:
import requests
import folium
import pandas as pd
from geopy.geocoders import Nominatim
from flask import Flask

colors_dict = {'Organization':              'aqua',
                'Business':                 'blue',
                'Performance Venue':        'lime',
                'Gallery/Museum':           'pink', 
                'School/Education':         'gold',
                'Landmark/Public Space':    'green',
                'Event/Festival':           'red',
                'Artist(s) Studio/Workshop':'orange',
                'House of Worship':         'purple',
                'Library':                  'brown'}


def descriptionize(row):
    
    output = f"<b>{row['Name of the place/activity you wish to add to map']}</b> <br> <b>Category:</b> {row['Category - What is it?']} <br><b>Type:</b> {row['Type']} <br><b>Location:</b> {row['Location']} <br><b>Notes:</b> {row['Notes']}<br> <b>What do you love/appreciate about it?:</b><br>{row['What do you love/appreciate about it?']}"
    return output


def condense_duplicates(df):
    
    #df['Popup label'] = pd.Series(dtype='str')
    #df['Size'] = pd.Series(dtype='str')
    
    locations = []
    sizes = []
    droplist = []
    
    for i in range(len(df)):
            
            current_add = df.loc[i, 'Geolocated address']
            
            if current_add in locations:
                droplist.append('yes')
                
                target_row = df[df['Geolocated address'] == current_add].iloc[0]
                
                target_index = df[df['Geolocated address'] == current_add].index.tolist()[0]
                
                df.loc[target_index, 'Popup label'] += ('<br><br>' + descriptionize(df.iloc[i]))
                
                df.loc[target_index, 'Size'] += 2
                
                df.loc[target_index, 'Color'] = 'black'
                
            else:
                droplist.append('no')
                
                df.loc[i, 'Popup label'] = descriptionize(df.iloc[i])
                
                df.loc[i, 'Size'] = 10
                
                if df.loc[i, 'Type'] in colors_dict.keys():
                    df.loc[i, 'Color'] = colors_dict[df.loc[i, 'Type']]
                else:
                    df.loc[i, 'Color'] = 'gray'
            
            locations.append(current_add)
            
    df['Drop?'] = droplist
    
    drop_indexes = df[df['Drop?'] == 'yes'].index.tolist()
    
    print(drop_indexes)
    
    df = df.drop(drop_indexes).reset_index()        

    return df



def break_up_lines(response_df):

    for (columnName, columnData) in response_df.iteritems():
        for row in range(len(columnData)):
            cell = str(columnData.iloc[row])
            count = 0
            overall_count = 0
            if cell == None:
                response_df.at[row, columnName] = ' '
            for char in cell:
                if count >= 50 and char == ' ':
                    newstring = cell[:overall_count] + '<br>' + cell[overall_count:]
                    response_df.at[row, columnName] = newstring
                    cell = newstring
                    count = 0
                count +=1
                overall_count += 1
                
    return response_df



base_map = folium.Map(location=[41.26, -72.95], zoom_start=13.3)

url = 'https://docs.google.com/spreadsheets/d/1EyIJRnLrWkeAJuxLVBxO2eSBMfKiQ2aHYLS3PLvQLLs/export?format=csv'
r = requests.get(url, allow_redirects=True)

open('responses.csv', 'wb').write(r.content)

response_df = pd.read_csv("responses.csv")

geolocator = Nominatim(user_agent="Luke's mapping app", timeout=2)

lat_list = []
long_list = []
add_list = []

for address in response_df['Location']:
    print(address)
    geo = geolocator.geocode(address)
    if geo != None:
        print(geo)
        lat = geo.latitude
        long = geo.longitude
        lat_list.append(lat)
        long_list.append(long)
        add_list.append(geo)
    else:
        lat_list.append('41.2705')
        long_list.append('-72.9470')
        add_list.append('West Haven, CT')
    
#break up large lines
    
    
response_df['Latitude'] = lat_list
response_df['Longitude'] = long_list
response_df['Geolocated address'] = add_list

response_df = break_up_lines(response_df)

response_df = condense_duplicates(response_df)

for index, row in response_df.iterrows():
    
    caption_info = row['Popup label']
    
    folium.CircleMarker(
                # Latitude, longitude for each marker
                location=[row['Latitude'], row['Longitude']],
        
                # Size, fill, color of the circle marker
                
                    radius = row['Size'], fill=True, color=row['Color'], #(row['FY18.Max.Population.Count']+1)/25, fill = True, color='orange',
        
                # Text that goes into the popup or tooltip
                tooltip = caption_info 
                  
                 ).add_to(base_map)


html = base_map._repr_html_()

file = open("index.html","w")
file.write(html)
file.close()


nan
Nanno, Ville d'Anaunia, Comunità della Val di Non, Provincia di Trento, Trentino-Alto Adige/Südtirol, 38012, Italia
nan
Nanno, Ville d'Anaunia, Comunità della Val di Non, Provincia di Trento, Trentino-Alto Adige/Südtirol, 38012, Italia
500 orange ave, west haven ct
500, Orange Avenue, Allingtown, West Haven, New Haven County, Connecticut, 06516, United States
The entire shoreline 
300 Elm Street West Haven, CT 
300, Elm Street, West Haven, New Haven County, Connecticut, 06516, United States
221 Bull Hill Ln West Haven, CT 06516
221, Bull Hill Lane, West Haven, New Haven County, Connecticut, 06516, United States
375 Morgan Ln Suite 408 West Haven, CT 06516
popups, various
popups, various
Corner of Campbell Ave and Main St  on the Green
86 Lake St West Haven, CT 06516
86, Lake Street, West Shore, West Haven, New Haven County, Connecticut, 06516, United States
Old Grove Park, Captain Thomas Blvd, West Haven, CT 06516
311 Shingle Hill Rd West Haven, CT 06516
311, Shingle Hill Road, Sea

In [8]:
response_df

,Timestamp,Name of the place/activity you wish to add to map,Category - What is it?,Type,Location,Notes,"Explain ""Other,"" if selected",Unnamed: 7,"Explain ""Other,"" if selected.1",Unnamed: 9,...,Address (please be a current West Haven resident),Preferred form of contact (text or email),Email Address,What do you love/appreciate about it?,Website / Social Media Profile?,Latitude,Longitude,Geolocated address,Popup label,Size
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,46.3145,11.048,"(Nanno, Ville d'Anaunia, Comunità della Val di...",NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,46.3145,11.048,"(Nanno, Ville d'Anaunia, Comunità della Val di...",NaN,NaN
2,7/5/2021 20:00:41,ESUMS,"Technology (gaming, web design...)",School/Education,"500 orange ave, west haven ct",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,41.2905,-72.9665,"(500, Orange Avenue, Allingtown, West Haven, N...",NaN,NaN
3,7/8/2021 10:43:59,"Savin Rock Surf Shop, the beach, and boardwalk","Visual Arts, Literary Arts, Craft/Design, Cult...",Landmark/Public Space,The entire shoreline,NaN,NaN,NaN,NaN,NaN,...,16 Smith street,Text,liz.johnston.11@gmail.com,My whole life my family emphasized value of na...,NaN,41.2705,-72.9470,"West Haven, CT",NaN,NaN
4,7/8/2021 14:53:27,West Haven Public Library,NaN,Library,"300 Elm Street West Haven, CT",NaN,NaN,NaN,NaN,NaN,...,300 Elm Street,abanks@westhavenlibrary.org,abanks@westhavenlibrary.org,NaN,westhavenlibrary.org,41.2756,-72.9521,"(300, Elm Street, West Haven, New Haven County...",NaN,NaN
5,7/9/2021 8:09:49,"Carizma Body Alliance, LLC","Dance, Culture, Heritage, Tradition",Business,"221 Bull Hill Ln West Haven, CT 06516",Brand new in 2021~!,NaN,NaN,NaN,NaN,...,NaN,NaN,artswestct@gmail.com,"African Movement, Hip Hop, Lyrical Contemporar...",https://www.facebook.com/Carizma-Body-Alliance...,41.2736,-72.9853,"(221, Bull Hill Lane, West Haven, New Haven Co...",NaN,NaN
6,7/9/2021 8:12:54,Top Hat Dance Academy,Dance,Business,"375 Morgan Ln Suite 408 West Haven, CT 06516",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,artswestct@gmail.com,"many styles, age 3+ no experience necessary",https://www.facebook.com/Top-Hat-Dance-Academy...,41.2705,-72.9470,"West Haven, CT",NaN,NaN
7,7/9/2021 8:18:39,Treasure Beads,"Visual Arts, Textile/Apparel, Craft/Design, Cu...",Business,"popups, various",Founded by Antionette Cameron (Twoni Cisse on ...,NaN,NaN,NaN,NaN,...,NaN,NaN,artswestct@gmail.com,waist beads to enhance self care in the Africa...,https://www.shoptreasurebeads.com/,41.2705,-72.9470,"West Haven, CT",NaN,NaN
8,7/9/2021 8:32:37,Your Queer Plant Shop,"Craft/Design, Culture, Heritage, Tradition, Ho...",Business,"popups, various",Founded by Anika Stewart and Georgina Marie - ...,NaN,NaN,NaN,NaN,...,NaN,NaN,artswestct@gmail.com,hard-to-find houseplants grown with artisanal...,https://www.facebook.com/yourqueerplantshop,41.2705,-72.9470,"West Haven, CT",NaN,NaN
9,7/9/2021 8:42:22,West Haven Farmers Market,"Craft/Design, Culinary Arts, Culture, Heritage...",Event/Festival,Corner of Campbell Ave and Main St on the Green,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,artswestct@gmail.com,Thurs and Sat on West Haven Center Green,https://www.facebook.com/WestHavenFarmersMarket,41.2705,-72.9470,"West Haven, CT",NaN,NaN


In [ ]:
if row['Geolocated address']
    droplist.append('yes')

In [85]:
def descriptionize(row):
    
    output = f"<b>{row['Name of the place/activity you wish to add to map']}</b> <br> <b>Category:</b> {row['Category - What is it?']} <br><b>Type:</b> {row['Type']} <br><b>Location:</b> {row['Location']} <br><b>Notes:</b> {row['Notes']}<br> <b>What do you love/appreciate about it?:</b><br>{row['What do you love/appreciate about it?']}"
    return output

In [87]:
def condense_duplicates(df):
    
    #df['Popup label'] = pd.Series(dtype='str')
    #df['Size'] = pd.Series(dtype='str')
    
    locations = []
    sizes = []
    droplist = []
    
    for i in range(len(df)):
            
            current_add = df.loc[i, 'Geolocated address']
            
            if current_add in locations:
                droplist.append('yes')
                
                target_row = df[df['Geolocated address'] == current_add].iloc[0]
                
                target_index = df[df['Geolocated address'] == current_add].index.tolist()[0]
                
                df.loc[target_index, 'Popup label'] += ('<br><br>' + descriptionize(df.iloc[i]))
                
                df.loc[target_index, 'Size'] += 2
                
                df.loc[target_index, 'Color'] = 'black'
                
            else:
                droplist.append('no')
                
                df.loc[i, 'Popup label'] = descriptionize(df.iloc[i])
                
                df.loc[i, 'Size'] = 10
                
                if df.loc[i, 'Type'] in colors_dict.keys():
                    df.loc[i, 'Color'] = colors_dict[df.loc[i, 'Type']]
                else:
                    df.loc[i, 'Color'] = 'gray'
            
            locations.append(current_add)
            
    df['Drop?'] = droplist
    
    drop_indexes = df[df['Drop?'] == 'yes'].index.tolist()
    
    print(drop_indexes)
    
    df = df.drop(drop_indexes).reset_index()        

    return df

In [76]:
condense_duplicates(response_df)

[1, 6, 7, 8, 9, 11, 15, 18, 21, 25, 26, 27, 34, 35, 37, 41, 45, 47, 49]


,index,Timestamp,Name of the place/activity you wish to add to map,Category - What is it?,Type,Location,Notes,"Explain ""Other,"" if selected",Unnamed: 7,"Explain ""Other,"" if selected.1",...,Preferred form of contact (text or email),Email Address,What do you love/appreciate about it?,Website / Social Media Profile?,Latitude,Longitude,Geolocated address,Popup label,Size,Drop?
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,46.3145,11.048,"(Nanno, Ville d'Anaunia, Comunità della Val di...",<b>nan</b> <br> <b>Category:</b> nan <br><b>Ty...,12,no
1,2,7/5/2021 20:00:41,ESUMS,"Technology (gaming, web design...)",School/Education,"500 orange ave, west haven ct",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,41.2905,-72.9665,"(500, Orange Avenue, Allingtown, West Haven, N...",<b>ESUMS</b> <br> <b>Category:</b> Technology ...,10,no
2,3,7/8/2021 10:43:59,"Savin Rock Surf Shop, the beach, and boardwalk","Visual Arts, Literary Arts, Craft/Design, Cult...",Landmark/Public Space,The entire shoreline,NaN,NaN,NaN,NaN,...,Text,liz.johnston.11@gmail.com,My whole life my family emphasized value of na...,NaN,41.2705,-72.9470,"West Haven, CT","<b>Savin Rock Surf Shop, the beach, and boardw...",28,no
3,4,7/8/2021 14:53:27,West Haven Public Library,NaN,Library,"300 Elm Street West Haven, CT",NaN,NaN,NaN,NaN,...,abanks@westhavenlibrary.org,abanks@westhavenlibrary.org,NaN,westhavenlibrary.org,41.2756,-72.9521,"(300, Elm Street, West Haven, New Haven County...",<b>West Haven Public Library</b> <br> <b>Categ...,10,no
4,5,7/9/2021 8:09:49,"Carizma Body Alliance, LLC","Dance, Culture, Heritage, Tradition",Business,"221 Bull Hill Ln West Haven, CT 06516",Brand new in 2021~!,NaN,NaN,NaN,...,NaN,artswestct@gmail.com,"African Movement, Hip Hop, Lyrical Contemporar...",https://www.facebook.com/Carizma-Body-Alliance...,41.2736,-72.9853,"(221, Bull Hill Lane, West Haven, New Haven Co...","<b>Carizma Body Alliance, LLC</b> <br> <b>Cate...",10,no
5,10,7/9/2021 8:45:38,The Cookie Haven,Culinary Arts,Business,"86 Lake St West Haven, CT 06516",NaN,NaN,NaN,NaN,...,NaN,artswestct@gmail.com,well executed and colorful themed cookie trays...,https://www.facebook.com/thecookiehavenct,41.2453,-72.9724,"(86, Lake Street, West Shore, West Haven, New ...",<b>The Cookie Haven</b> <br> <b>Category:</b> ...,10,no
6,12,7/9/2021 8:54:37,Connecticut Poetry Society,Literary Arts,Organization,"311 Shingle Hill Rd West Haven, CT 06516",NaN,NaN,NaN,NaN,...,NaN,artswestct@gmail.com,"President Emeritus, Tony Fusco, Poet Laureate ...",https://www.facebook.com/Connecticut-Poetry-So...,41.2567,-72.9763,"(311, Shingle Hill Road, Sea Bluff, West Haven...",<b>Connecticut Poetry Society</b> <br> <b>Cate...,10,no
7,13,7/9/2021 8:57:17,Verge Arts Group,"Multidisciplinary, business development, marke...",Business,"18 4th Ave West Haven, CT 06516",NaN,NaN,NaN,NaN,...,NaN,artswestct@gmail.com,"one on one, customized support for artists and...",https://www.facebook.com/vergearts,41.278,-72.9477,"(18, 4th Avenue, West Haven, New Haven County,...",<b>Verge Arts Group</b> <br> <b>Category:</b> ...,10,no
8,14,7/9/2021 9:00:25,ArtsWestCT,"Multidisciplinary, Arts Service Organization",Organization,"304 Center St West Haven, CT 06516",NaN,NaN,NaN,NaN,...,NaN,artswestct@gmail.com,a community that honors artists' valuable plac...,https://www.facebook.com/Westhavenarts,41.273,-72.9513,"(304, Center Street, West Haven, New Haven Cou...",<b>ArtsWestCT</b> <br> <b>Category:</b> Multid...,10,no
9,16,7/9/2021 9:10:51,Z's Corner Cafe,"Comedy, Music",Performance Venue,"407 Campbell Ave West Haven, CT 06516",NaN,NaN,NaN,NaN,...,NaN,artswestct@gmail.com,iconic rock venue for 24 years,https://www.facebook.com/Zs-Corner-Cafe-617368...,41.2696,-72.9488,"(407, Campbell Avenue, Savin Rock, West Haven,...",<b>Z's Corner Cafe</b> <br> <b>Category:</b> C...,10,no


In [22]:
for index, row in response_df.iterrows():

    current_geo = str(row['Geolocated address'])
    #with_same_address = response_df.filter(like=current_geo)
    with_same_address = response_df.loc[response_df['Geolocated address'] == current_geo]
    
    print(len(with_same_address))

    if len(with_same_address.index) == 1:

        caption_info = f"<b>{row['Name of the place/activity you wish to add to map']}</b> <br> <b>Category:</b> {row['Category - What is it?']} <br><b>Type:</b> {row['Type']} <br><b>Location:</b> {row['Location']} <br><b>Notes:</b> {row['Notes']}" 
    
        if row['Type'] in colors_dict.keys():
    
            folium.CircleMarker(
                # Latitude, longitude for each marker
                location=[row['Latitude'], row['Longitude']],
        
                # Size, fill, color of the circle marker
                
                    radius = 10, fill=True, color=colors_dict[row['Type']], #(row['FY18.Max.Population.Count']+1)/25, fill = True, color='orange',
        
                # Text that goes into the popup or tooltip
                tooltip = caption_info 
                  
                 ).add_to(base_map)

        else:

         folium.CircleMarker(
                # Latitude, longitude for each marker
                location=[row['Latitude'], row['Longitude']],
        
                # Size, fill, color of the circle marker
                
                    radius = 10, fill=True, color='gray', #(row['FY18.Max.Population.Count']+1)/25, fill = True, color='orange',
        
                # Text that goes into the popup or tooltip
                tooltip = caption_info 
                  
                 ).add_to(base_map)

    elif len(with_same_address.index) > 1:

        caption_info = ''
        
        print(with_same_address)

        for index, row in with_same_address.iterrows():
            
            

            caption_info += f"<b>{row['Name of the place/activity you wish to add to map']}</b> <br> <b>Category:</b> {row['Category - What is it?']} <br><b>Type:</b> {row['Type']} <br><b>Location:</b> {row['Location']} <br><b>Notes:</b> {row['Notes']} <br> <br>"
        
        folium.CircleMarker(
                # Latitude, longitude for each marker
                location=[row['Latitude'], row['Longitude']],
        
                # Size, fill, color of the circle marker
                
                    radius = 10+(len(with_same_address.index)), fill=True, color='black', #(row['FY18.Max.Population.Count']+1)/25, fill = True, color='orange',
        
                # Text that goes into the popup or tooltip
                tooltip = caption_info 
                  
                 ).add_to(base_map)



html = base_map._repr_html_()

file = open("index.html","w")
file.write(html)
file.close()

0
0
0
10
            Timestamp Name of the place/activity you wish to add to map  \
3   7/8/2021 10:43:59   Savin Rock Surf Shop, the beach, and boardwalk    
6    7/9/2021 8:12:54                            Top Hat Dance Academy    
7    7/9/2021 8:18:39                                    Treasure Beads   
8    7/9/2021 8:32:37                             Your Queer Plant Shop   
9    7/9/2021 8:42:22                         West Haven Farmers Market   
11   7/9/2021 8:52:32             West Haven Center for Coastal Ecology   
15   7/9/2021 9:09:00                             West Haven Hoop Guild   
21  7/9/2021 19:11:15                        UNH Chargers Marching Band   
41  7/9/2021 20:28:10                                 Ralph Eberle Park   
45  7/9/2021 20:38:41                                          Sri Thai   

                               Category - What is it?  \
3   Visual Arts, Literary Arts, Craft/Design, Cult...   
6                                               Dan

In [6]:
len(with_same_address)

0

In [23]:
base_map